In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataPreprocessingConfig():
    root_dir: Path
    input_data_dir: Path
    preprocessed_audios_dir: Path
    dataframe_dir: Path
    num_samples: int
    max_overlays: int
    metadata_mapping: dict
    

In [2]:
from Audio_Vision.utils.common import read_yaml, create_directories
from Audio_Vision import logger

In [11]:
class ConfigurationManager():
    def __init__(self):
        self.config_filepath = Path("config/config.yaml")
        self.params_filepath = Path("params.yaml")

        self.config = read_yaml(self.config_filepath)
        self.params = read_yaml(self.params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.data_preprocessing
        params = self.params

        create_directories([config.root_dir])

        data_preprocessing_config = DataPreprocessingConfig(
            root_dir =  config.root_dir,
            input_data_dir = config.input_data_dir,
            preprocessed_audios_dir = config.preprocessed_audios_dir,
            dataframe_dir = config.dataframe_dir,
            num_samples = params.num_samples,
            max_overlays = params.max_overlays,
            metadata_mapping = params.metadata_mapping
        )
        return data_preprocessing_config

In [13]:
! pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)


In [12]:
import os
import random
import pandas as pd
from pydub import AudioSegment

In [25]:
class DataPreprocessing():
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config

    def data_preprocessing(self):
        
        data_dir = self.config.input_data_dir  
        output_dir = self.config.preprocessed_audios_dir
        dataframe_dir = self.config.dataframe_dir
 
        metadata_mapping = self.config.metadata_mapping
        all_audio_files = os.listdir(data_dir)
        class_audio_map = {}
        for file in all_audio_files:
            if file.endswith(".wav"):
                class_id = file.split("_")[0]  
                class_audio_map.setdefault(class_id, []).append(file)

        num_samples = self.config.num_samples  
        max_overlays = self.config.max_overlays  

        data = []
        for i in range(num_samples):
            num_overlay = random.randint(2, max_overlays)  
            selected_classes = random.sample(list(class_audio_map.keys()), num_overlay)  

            mixed_audio = None
            labels_numeric = []
            labels_text = []

            for cls in selected_classes:
                audio_file = random.choice(class_audio_map[cls])  
                audio_path = os.path.join(data_dir, audio_file)
                sound = AudioSegment.from_wav(audio_path)  
                if mixed_audio is None:
                    mixed_audio = sound  
                else:
                    mixed_audio = mixed_audio.overlay(sound)  

                labels_numeric.append(cls) 
                labels_text.append(metadata_mapping.get(cls, "unknown"))  

            output_filename = f"mixed_{i}.wav"
            output_path = os.path.join(output_dir, output_filename)
            create_directories([output_dir]) 
            mixed_audio.export(output_path, format="wav")
            data.append({"file_path": output_path, "labels_numeric": labels_numeric, "labels_text": labels_text})

        df = pd.DataFrame(data)
        df.to_csv(os.path.join(dataframe_dir, "preprocessed_data.csv"), index=False)
        logger.info("Multi-label dataset with metadata mapping created successfully!")


In [26]:
%pwd

'c:\\Users\\kanis\\Desktop\\Audio_Classification'

In [27]:
os.chdir(r"c:\\Users\\kanis\\Desktop\\Audio_Classification")

In [28]:
try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_data_preprocessing_config()
    data_preprocessing = DataPreprocessing(config = data_preprocessing_config)
    data_preprocessing.data_preprocessing()
except Exception as e:
    raise e

[2025-02-01 02:42:24,190]: INFO: common: yaml file: config\config.yaml loaded successfully
[2025-02-01 02:42:24,197]: INFO: common: yaml file: params.yaml loaded successfully
[2025-02-01 02:47:12,744]: INFO: 2416190755: Multi-label dataset with metadata mapping created successfully!
